In [3]:
import numpy as np
from bert_serving.client import BertClient
from termcolor import colored

train_data = [
    (u"That was very bad", {"cats": {"POSITIVE": 0, "NEGATIVE": 1}}),
    (u"it is so bad", {"cats": {"POSITIVE": 0, "NEGATIVE": 1}}),
    (u"so terrible", {"cats": {"POSITIVE": 0, "NEGATIVE": 1}}),
    (u"I like it", {"cats": {"POSITIVE": 1, "NEGATIVE": 0}}),
    (u"It is very good.", {"cats": {"POSITIVE": 1, "NEGATIVE": 0}}),
    (u"That was great!", {"cats": {"POSITIVE": 1, "NEGATIVE": 0}})
]
questions = [t[0] for t in train_data]
print(questions)

['That was very bad', 'it is so bad', 'so terrible', 'I like it', 'It is very good.', 'That was great!']


In [48]:
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

array([[ 0.13186465,  0.32404128, -0.82704467, ..., -0.3711961 ,
        -0.3925013 , -0.31721827],
       [ 0.24873514, -0.12334406, -0.38933834, ..., -0.44756237,
        -0.5591354 , -0.11345179],
       [ 0.28627363, -0.18580145, -0.30906814, ..., -0.29593706,
        -0.39310458,  0.07640193]], dtype=float32)

In [51]:
bc.encode(['then do it better', 'I told you not to tell anyone.'])

array([[ 0.28627363, -0.18580145, -0.30906814, ..., -0.29593706,
        -0.39310458,  0.07640193],
       [ 0.38893676, -0.38931534, -0.30416116, ...,  0.14372948,
         0.04086732,  0.00814748]], dtype=float32)

In [66]:
a=bc.encode(['First do it', 'then do it right'])
b=bc.encode(['then do it better'])       
np.vstack((a, b))

array([[ 0.13186465,  0.32404128, -0.82704467, ..., -0.3711961 ,
        -0.3925013 , -0.31721827],
       [ 0.24873514, -0.12334406, -0.38933834, ..., -0.44756237,
        -0.5591354 , -0.11345179],
       [ 0.2862737 , -0.18580146, -0.30906788, ..., -0.29593676,
        -0.3931048 ,  0.07640249]], dtype=float32)

In [32]:
lst_str=['First do it', 'then do it right', 'then do it better']
list_vec = bc.encode(lst_str)
list_label = [0 for _ in lst_str]  # a dummy list of all-zero labels
print(list_label)
print(type(list_vec), list_vec)

[0, 0, 0]
<class 'numpy.ndarray'> [[ 0.13186485  0.32404107 -0.82704383 ... -0.37119558 -0.39250153
  -0.31721818]
 [ 0.24873503 -0.12334415 -0.38933882 ... -0.44756237 -0.55913574
  -0.11345189]
 [ 0.28627354 -0.18580157 -0.30906835 ... -0.29593667 -0.3931053
   0.07640244]]


In [25]:
import tensorflow as tf
# write to tfrecord
with tf.python_io.TFRecordWriter('tmp.tfrecord') as writer:
    def create_float_feature(values):
        return tf.train.Feature(float_list=tf.train.FloatList(value=values))

    def create_int_feature(values):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))

    for (vec, label) in zip(list_vec, list_label):
        features = {'features': create_float_feature(vec), 'labels': create_int_feature([label])}
        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())

In [26]:
def _decode_record(record):
    """Decodes a record to a TensorFlow example."""
    return tf.parse_single_example(record, {
        'features': tf.FixedLenFeature([768], tf.float32),
        'labels': tf.FixedLenFeature([], tf.int64),
    })

ds = (tf.data.TFRecordDataset('tmp.tfrecord').repeat().shuffle(buffer_size=100).apply(
    tf.contrib.data.map_and_batch(lambda record: _decode_record(record), batch_size=64))
      .make_one_shot_iterator().get_next())


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.


In [30]:
# print(ds)
# name_to_features = {
#     "feature": tf.FixedLenFeature([max_seq_length * num_hidden], tf.float32),
#     "label_ids": tf.FixedLenFeature([], tf.int64),
# }

# def _decode_record(record, name_to_features):
#     """Decodes a record to a TensorFlow example."""
#     example = tf.parse_single_example(record, name_to_features)
#     example['feature'] = tf.reshape(example['feature'], [max_seq_length, -1])
#     return example

In [5]:
# As a feature of BERT, you may get encodes of a pair of sentences by concatenating them with |||, e.g.
bc.encode(['First do it ||| then do it right'])

array([[-5.04763722e-02, -4.87895310e-02, -4.83554065e-01,
        -3.42728883e-01, -1.14914678e-01,  1.62609816e-01,
         2.15244770e-01,  3.63935769e-01,  3.03730786e-01,
        -4.82231468e-01, -4.92257215e-02, -1.86317861e-01,
        -1.67600513e-01, -2.55926222e-01, -9.69771028e-01,
         6.47908509e-01, -3.97496045e-01, -2.62127548e-01,
         3.59476879e-02,  2.64713436e-01, -2.11805508e-01,
         3.13155830e-01, -4.49802876e-01, -7.70862550e-02,
         2.49953300e-01, -3.49985540e-01,  1.01401851e-01,
        -4.91841704e-01, -1.98673695e-01,  2.33608961e-01,
         3.21058810e-01, -2.94979453e-01, -1.78063601e-01,
        -1.19715571e-01,  1.49329379e-01,  4.04063702e-01,
         4.29855198e-01,  7.30635673e-02, -4.05314118e-01,
         6.16669953e-02, -5.23368716e-01, -3.88182312e-01,
        -2.52037108e-01, -4.18566853e-01, -1.22309744e-01,
        -3.84358525e-01, -6.28781468e-02,  7.63056800e-02,
         3.56706455e-02, -5.75758040e-01, -8.97076726e-0

In [8]:
doc_vecs = bc.encode(questions)
topk = 5

In [10]:
def get_similar(query):
    query_vec = bc.encode([query])[0]
    score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
    print('top %d questions similar to "%s"' % (topk, colored(query, 'green')))
    for idx in topk_idx:
        print('> %s\t%s' % (colored('%.1f' % score[idx], 'cyan'), colored(questions[idx], 'yellow')))

query="That was very bad"
get_similar(query)

top 5 questions similar to "That was very bad"
> 11.3	That was very bad
> 8.2	it is so bad
> 7.9	That was great!
> 7.7	It is very good.
> 7.6	so terrible


In [44]:
from sagas.nlu.corpus_helper import filter_term, lines, divide_chunks
import numpy
import spacy

samples_count=1000
# dataf = "/pi/ai/seq2seq/fra-eng-2019/fra.txt"
dataf='/pi/ai/seq2seq/jpn-eng-2019/jpn.txt'
print('loading corpus ...')
pairs = lines(dataf)
total = len(pairs)
print('total', total)
array = numpy.array(pairs)
random_rows = numpy.random.randint(total, size=samples_count)
# print('pickup', random_rows)
# print(array[random_rows, :])

# print('analyse ...')
rows = array[random_rows, :]
dataset = []
for r in rows:
    sents=str(r[0])
    tr_lang=str(r[1].strip())
    dataset.append(sents)

print('pickup ok, total %d'%len(dataset))

loading corpus ...
total 43953
pickup ok, total 1000


In [17]:
questions=dataset
doc_vecs = bc.encode(questions)

query="I want the two of you to quit arguing."
get_similar(query)

top 5 questions similar to "I want the two of you to quit arguing."
> 8.9	I know what the three of you want.
> 8.4	I told you to leave.
> 8.2	It is not appropriate for you to do this.
> 8.2	I want you and me to be happy.
> 8.2	You're wasting both of our time.


In [19]:
query="I told you to leave."
get_similar(query)

top 5 questions similar to "I told you to leave."
> 10.5	I told you to leave.
> 8.6	They want you to resign.
> 8.5	I knew you wouldn't help us.
> 8.4	We're waiting for you to leave.
> 8.2	I now know what I told you was wrong.


In [34]:
import numpy as np
print(len(doc_vecs))
np.save('./data/bert/samples_1000.npy', doc_vecs) # save

1000


In [35]:
doc_vecs=np.load('./data/bert/samples_1000.npy') # load
print(len(doc_vecs))

1000


In [36]:
query="I told you to leave."
get_similar(query)

top 5 questions similar to "I told you to leave."
> 10.5	I told you to leave.
> 8.6	They want you to resign.
> 8.5	I knew you wouldn't help us.
> 8.4	We're waiting for you to leave.
> 8.2	I now know what I told you was wrong.


In [37]:
doc_vecs=list_vec
query="I told you to leave."
get_similar(query)

top 5 questions similar to "I told you to leave."
> 6.9	Do you know how Tom got the scar on his chin?
> 6.8	I don't like those people.
> 6.5	Tom taught Mary to swim.


In [38]:
doc_vecs=np.load('./data/bert/samples_1000.npy') # load
query="I told you to leave."
get_similar(query)

top 5 questions similar to "I told you to leave."
> 10.5	I told you to leave.
> 8.6	They want you to resign.
> 8.5	I knew you wouldn't help us.
> 8.4	We're waiting for you to leave.
> 8.2	I now know what I told you was wrong.


In [45]:
questions=[]
for r in array:
    sents=str(r[0])
    questions.append(sents)
print(len(questions))
print(questions[:5])

43953
['Go.', 'Go.', 'Hi.', 'Hi.', 'Run.']


In [46]:
from bert_serving.client import BertClient
import numpy as np

# 默认的选项: "max_seq_len=25"
# bc = BertClient(check_length=False) # disable the length-check, or, start a new server with a larger "max_seq_len"
bc = BertClient()
doc_vecs = bc.encode(questions)
np.save('./data/bert/jpn-eng.npy', doc_vecs) # save
print('done')

done


In [52]:
doc_vecs=np.load('./data/bert/jpn-eng.npy') # load
query="I told you to leave."
get_similar(query)

top 5 questions similar to "I told you to leave."
> 9.1	I told you not to tell anyone.
> 9.0	I thought I told you to go to bed.
> 8.9	I thought I told you to stay in the car.
> 8.9	I said stay back.
> 8.9	I thought I told you to clean your room.


In [60]:
query="I want a book."
get_similar(query)

top 5 questions similar to "I want a book."
> 11.8	I want a book.
> 11.1	I want a computer.
> 11.1	I want a guitar.
> 10.8	I want a personal computer.
> 10.8	I want something to read.


In [63]:
print(json.dumps(bc.server_status, indent=2, ensure_ascii=False))

{
  "client": "622011fc-2a47-4566-a19e-b691f660c1c7",
  "num_process": 3,
  "ventilator -> worker": [
    "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpKwF5i1/socket",
    "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpQ3zt4o/socket",
    "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpD5ZyQI/socket",
    "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpteIJ8y/socket",
    "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpac7Bg9/socket",
    "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpMQlvcG/socket",
    "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmp8QmPJM/socket",
    "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpAtuwba/socket"
  ],
  "worker -> sink": "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmpxVOUkN/socket",
  "ventilator <-> sink": "ipc:///var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmphH8qe9/socket",
  "server_current_time": "2019-06-16 21:37:43.038621",
  "statistic

In [65]:
loop_for=2
for n in range(loop_for):
    print(n)

0
1


In [71]:
from bert_serving.client import BertClient
# bc = BertClient(ip='192.168.1.105')
# bc = BertClient(ip='pc')
bc = BertClient(ip='localhost')
bc.encode(['First do it', 'then do it right', 'then do it better'])

array([[ 0.13186465,  0.32404128, -0.82704467, ..., -0.3711961 ,
        -0.3925013 , -0.31721827],
       [ 0.24873514, -0.12334406, -0.38933834, ..., -0.44756237,
        -0.5591354 , -0.11345179],
       [ 0.28627363, -0.18580145, -0.30906814, ..., -0.29593706,
        -0.39310458,  0.07640193]], dtype=float32)